In [ ]:
#! pip install transformers

In [ ]:
#!pip install torchtext

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset


from sklearn.datasets import fetch_20newsgroups

import spacy

import numpy as np

import random
import math
import time

from itertools import chain, islice

import pandas as pd

from tqdm import tqdm

import json

SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

! python -m spacy download en

spacy_en = spacy.load('en')

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>', 
            lower = True)

# should be 0 if the sentence is natural and 1 if this is encoded
LABEL = LabelField(dtype = torch.float)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
data_size = 1000  #size of generated dataset

In [4]:
#IMDB dataset
dataset_name = "imdb"
train_data, test_data = IMDB.splits(SRC, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [26]:
# for News dataset
newsgroups_train = fetch_20newsgroups(subset='train')

In [27]:
dataset_name = "news"

train_data = []
for i, text in enumerate(newsgroups_train["data"]):
    if "Subject:" in text:
        poss_text = text.split("Subject:")[1]
        if "Re:" in text:
            train_data.append(poss_text.split("Re:")[1])
        else:    
            train_data.append(poss_text)
    if len(train_data) == data_size:
        break

In [8]:
#Twitter

dataset_name = "twitter"

data = pd.read_csv("../data/texts/twitter.csv", encoding = "ISO-8859-1")

data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [13]:
train_data = data.SentimentText

Generating data

In [14]:
file_names = ["FLC_16.json", "FLC_32.json", "FLC_64.json", 
              "VLC_16.json", "VLC_32.json", "VLC_64.json",
              "FLC_2.json", "FLC_4.json", "FLC_8.json", 
              "VLC_2.json", "VLC_4.json", "VLC_8.json"]

In [15]:
#writing the record in json file 
def write_json(data, filename='data.json'): 
    with open(filename,'a') as f: 
        f.write(json.dumps(data)+"\n")

In [16]:
dir_path = "./data/"

In [ ]:
#adding original texts for IMDB
for text in tqdm(islice(train_data.text, 0, data_size)):
    for name in file_names:
        write_json({"text": " ".join(text), "label":1}, dir_path + dataset_name + "_attacker_data" + name)

In [34]:
#adding original texts for NEWS
for text in tqdm(islice(train_data, 0, data_size)):
    for name in file_names:
        write_json({"text": text, "label":1}, dir_path + dataset_name + "_attacker_data" + name)

1000it [00:09, 107.79it/s]


In [17]:
#adding original texts for Twitter
for text in tqdm(islice(train_data, 0, data_size)):
    for name in file_names:
        write_json({"text": text.strip(), "label":1}, dir_path + dataset_name + "_attacker_data" + name)

1000it [00:08, 118.14it/s]


In [18]:
#adding generated texts to json files
for name in file_names:
    with open("./texts/" + name) as json_file: #../data/texts/
        data = json.load(json_file)

    for text in data:
        write_json({"text": text, "label":0},
                 dir_path + dataset_name + "_attacker_data" + name)